In [2]:
from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.common.data_environment import DataEnvironment
from thetaray.api.graph import publish_nodes, publish_edges
from thetaray.api.evaluation import read_alerted_activities, load_evaluated_activities
import random
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import *
import pandas as pd
from thetaray.common import Constants

In [3]:
import datetime
from dateutil.relativedelta import relativedelta
import json
import logging
from pyspark.sql import DataFrame, Window, functions as f
from pyspark.sql import SQLContext
from pyspark.sql.types import LongType
import yaml

from common.libs import dates as dates_lib
from common.libs import features_discovery
from common.libs.features_executor import FeaturesExecutor
from common.libs.feature_engineering import max_look_back_monthly_features, max_look_back_daily_weekly_features
from common.libs.zscore import enrich_with_z_score
from common.factory.wrangling_execution_strategy import get_wrangling_execution_strategy
from common.factory.eval_flow_definition import get_evaluation_flow_definition
from common.factory.domain_definition import get_domain_definition
from common.notebook_utils.wrangling.wrangling_execution_strategy import WranglingExecutionStrategy
from common.definitions.domain import DomainDefinition
from common.definitions.eval_flow import EvaluationFlowDefinition
from common.libs.context_utils import get_dataset

from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.api.solution import IngestionMode
from thetaray.common import Constants
from thetaray.common.data_environment import DataEnvironment

logging.getLogger().handlers[0].setFormatter(logging.Formatter(fmt='%(levelname)s: %(asctime)s @ %(message)s',datefmt='%Y-%m-%d %H:%M:%S'))
logging.basicConfig(level=logging.INFO)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


from thetaray.api.context import init_context
import datetime
from thetaray.common import Constants

from common.libs.config.loader import load_config
from common.libs.config.basic_execution_config_loader import BasicExecutionConfig, DevBasicExecutionConfig
from common.libs.context_utils import is_run_triggered_from_airflow



with open('/thetaray/git/solutions/domains/demo_fuib/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']

execution_date=datetime.datetime(1970, 1, 1)

context = init_context(domain='demo_fuib',
                       execution_date=execution_date,
                       spark_conf=spark_config,
                       spark_master='local[*]',
                       allow_type_changes=True)

spark = context.get_spark_session()
sc = SQLContext(spark)
params = context.parameters
print(f"Spark UI URL: {context.get_spark_ui_url()}")

print(json.dumps(params, indent=4))

/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-09-09 11:44:15,508:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-09-09 11:44:15,518:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-09-09 11:44:15,533:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_indiv_customer_insights, but data encryption is disabled in deployment
2025-09-09 11:44:15,666:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_digital_wallets_customer_insights, but data encryption is disabled in deployment
2025-09-09 11:44:15,680:WARNING:thetaray.common.logging:Encryption is enabled on dataset customer_insights, but data encryp

Added `alias` successfully.


INFO: 2025-09-09 11:44:52 @ === Finished updating schema for Evaluation Flows on Minio ===


Added `alias` successfully.
Spark UI URL: https://jupyterhub-platform-thetalab.sonar.thetaray.cloud/user/andre.gutnik/proxy/4040/jobs/
{}


/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


### Account Nodes

In [4]:
trx_df = dataset_functions.read(context, 'trx_enriched',from_job_ts = datetime.datetime(1970,1,1), data_environment=DataEnvironment.PUBLIC)
trx_df.count()
# trx_df = trx_df.withColumn('customer_id', f.explode(f.array('creditor_identification', 'debtor_identification')))
# trx_df = trx_df.withColumn('customer_name', f.when(f.col('customer_id') == f.col('creditor_identification'), f.col('creditor_name')).otherwise(f.col('debtor_name')))
# trx_df = trx_df.withColumn('customer_country_code', f.when(f.col('customer_id') == f.col('creditor_identification'), f.col('creditor_country_code')).otherwise(f.col('debtor_country_code')))

# customer_df = trx_df.groupBy('customer_id').agg(f.first('customer_name', ignorenulls=True).alias('NM'),
#                                                 f.first('customer_country_code', ignorenulls=True).alias('CT'),
#                                                f.first('delivery_timestamp',ignorenulls=True).alias('effective_date'))

# customer_df = customer_df.select('effective_date',
#                                  'CT',
#                                  'NM',
#                                  f.col('customer_id').alias('AN'),
#                                  f.col('customer_id').alias('id'))


600806

In [7]:
# when aggregating on the walled_id level, we must rename the columns to match the graph metadata NodeType fields identifiers ("CN","NM", "CT", etc)

customer_df = trx_df.groupBy('customer_id').agg(f.first('customer_id', ignorenulls=True).alias('AN'), #CN is actually not used. We will not select it 
                                                f.first('customer_name', ignorenulls=True).alias('NM'),
                                                f.first('cp_jurisdiction', ignorenulls=True).alias('CT'),
                                                f.first('txn_ts',ignorenulls=True).alias('effective_date'))

customer_df = customer_df.select('AN',
                                 'NM',
                                 'CT',
                                 'effective_date',
                                 f.col('customer_id').alias('id'))

customer_df = customer_df.na.drop(subset=["AN"])

customer_df.count()

1500

In [10]:
# when aggregating on the walled_id level, we must rename the columns to match the graph metadata NodeType fields identifiers ("CN","NM", "CT", etc)

cp_df = trx_df.groupBy('cp_id').agg(f.first('cp_id', ignorenulls=True).alias('AN'),
                                                f.first('counterparty_name', ignorenulls=True).alias('NM'),
                                                f.first('cp_jurisdiction', ignorenulls=True).alias('CT'),
                                                f.first('txn_ts',ignorenulls=True).alias('effective_date'))


cp_df = cp_df.select('AN',
                     'NM',
                     'CT',
                     'effective_date',
                     f.col('cp_id').alias('id'))

cp_df = cp_df.na.drop(subset=["AN"])

cp_df.count()

481575

In [11]:
final_df = customer_df.union(cp_df)
final_df.count()

483075

In [ ]:
publish_nodes(context=context,
              nodes_df=final_df,
              graph_identifier='graph',
              node_type='AC',
              data_environment=DataEnvironment.PUBLIC)

### Transaction Edges

In [ ]:
trx_df=trx_df.na.drop(subset=["creditor_identification",'debtor_identification'])

In [ ]:
edges_df = trx_df.select(
    f.col('transaction_id').alias('id'),
    f.col('').alias('effective_date'), #TODO: complete 
    f.col('').alias('AM'), #TODO: complete
    f.col('debtor_identification').alias('source_node'), 
    f.col('creditor_identification').alias('target_node'),
    f.col('trx_currency_code').alias('CR')
).withColumn('CT', f.lit(1).cast('long'))

In [ ]:
edges_df = edges_df.na.drop(subset=["id"])

In [ ]:
publish_edges(context=context,
              edges_df=edges_df,
              graph_identifier='graph',
              edge_type='', #TODO: complete 
              source_node_type='',  #TODO: complete 
              target_node_type='', #TODO: complete 
              data_environment=DataEnvironment.PUBLIC)

In [ ]:
context.close()